# RAG Universidad Nacional

* Francisco Josse Roja Rojas (frrojasr@unal.edu.co)
* Yeira Liseth Rodríguez Rodríguez (yrodriguezro@unal.edu.co)

A continuación se presenta el tercer ejercicio del capítulo 7 de la materia de Inteligencia Artificial y mini-robots

#### 1. Instalación de librería
Se intala la librería para usar tranformers

In [ ]:
!pip install faiss-cpu transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

#### 2. Cargar corpus de documentos
Debido a restricciones por parte de las páginas oficiales de la Universidad Nacional de Colombia, se crea el siguiente corpus de documentos para ilustrar el RAG

In [ ]:
documentos = [
  "La presión por aumentar la estatura del caballo criollo colombiano ha llevado al uso excesivo de hormonas y vitaminas, poniendo en riesgo su bienestar animal.",  # :contentReference[oaicite:1]{index=1}
  "En el Atlántico, científicos detectaron 49 variantes genéticas en personas de 45 a 70 años que podrían anticipar riesgos de Alzheimer, Parkinson u otras demencias.",  # :contentReference[oaicite:2]{index=2}
  "Un estudio concluye que la represión policial hacia consumidores de bazuco es ineficaz y vulnera derechos humanos, recomendando un enfoque de salud pública.",  # :contentReference[oaicite:3]{index=3}
  "Investigadores en Antioquia desarrollaron un sistema con inteligencia artificial y satélites para predecir zonas críticas de deforestación hasta con dos años de antelación.",  # :contentReference[oaicite:4]{index=4}
  "Se implementaron 29 aulas STEAM en regiones apartadas, incluyendo San Basilio de Palenque, para fortalecer la educación en ciencia y tecnología.",  # :contentReference[oaicite:5]{index=5}
  "La alimentación de gallinas criollas con semillas de lino aumentó significativamente los niveles de omega‑3 en los huevos, mejorando su valor nutricional.",  # :contentReference[oaicite:6]{index=6}
]


#### 3. FAISS
Vectorizar y construir el índice FAISS

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Cargar modelo de embeddings local
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

# Vectorizar el corpus
vectores = modelo_embeddings.encode(documentos, show_progress_bar=True)

# Crear el indice FAISS
index = faiss.IndexFlatL2(vectores.shape[1])
index.add(np.array(vectores))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### 4. Contextos
Función para recuperar múltiples contextos

In [ ]:
def buscar_contexto(pregunta, top_k=3):
    pregunta_vec = modelo_embeddings.encode([pregunta])
    _, indices = index.search(np.array(pregunta_vec), top_k)
    fragmentos = [documentos[i] for i in indices[0]]
    return " ".join(fragmentos)


#### 5. Generador
Cargar modelo generativo FLAN-T5 optimizado para QA

In [ ]:
from transformers import pipeline

# Modelo multilingue, sigue instrucciones y responde preguntas
generador = pipeline("text2text-generation", model="google/flan-t5-base")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


#### 6. Respuesta
Función de respuesta mejorada con prompt claro

In [ ]:
def responder(pregunta):
    contexto = buscar_contexto(pregunta, top_k=3)
    prompt = f"Contesta la siguiente pregunta usando solo la información del contexto.\n\nContexto: {contexto}\nPregunta: {pregunta}"
    resultado = generador(prompt, max_new_tokens=100)[0]['generated_text']
    return resultado.strip()

#### 7. Probar el sistema
Se prueban 3 preguntas en donde el sistema logra identificar el contexto pero por la falta de información no logra dar una buena respuesta, evidenciando una oportunidad de mejora en la coherencia y redacción de la respuesta.

In [ ]:
print(responder("¿Cuántas aulas STEM hay?"))

29


In [ ]:
print(responder("¿Puede alimentarse a la gallina con semillas de lino?"))

La alimentación de gallinas criollas con semillas de lino aumentó significativamente los niveles de omega3 en los huevos, mejorando su valor nutricional


In [ ]:
print(responder("¿Cómo la represión policial afecta a los consumidores?"))

ineficaz y vulnera derechos humanos
